## Pseudo code to show process of computing RV

#### Load data

In [ ]:
order = 65
excalibur_mask  = fits_data['EXCALIBUR_MASK'][order]
y           = fits_data['spectrum'][order][excalibur_mask]
y_err       = fits_data['uncertainty'][order][excalibur_mask]
continuum   = fits_data['continuum'][order][excalibur_mask]
x           = fits_data['BARY_EXCALIBUR'][order][excalibur_mask]

# Normalize intensity by continuum 
y = y/continuum
y_err = y_err/continuum

#### Compute shift for one match

In [ ]:
# x1, y1, y1_err : comes from feature1
# x2, y2, y2_err : comes from feature2

# Interp first file
f1 = interp1d(x1, y1, kind='cubic', fill_value="extrapolate")
f1_upper_err = interp1d(x1, y1 + y1_err, kind='cubic', fill_value="extrapolate")
f1_lower_err = interp1d(x1, y1 - y1_err, kind='cubic', fill_value="extrapolate")

# ChiSquare fit model:
def model_chi2(A):

    # Interpolate template
    interp_x2 = x2 * (1 + A/c)                                                  # Wavelengths are be stretched by a factor of (1 + v/c)
    f2 = interp1d(interp_x2, y2, kind='cubic', fill_value="extrapolate")

    # Find common x-range
    xmin = max([min(x1), min(interp_x2)])
    xmax = min([max(x1), max(interp_x2)])
    xnewCommon = np.linspace(xmin, xmax, interp_size)
    
    # Evaluate interpolation
    ynew1 = f1(xnewCommon)
    ynew2 = f2(xnewCommon)

    # Evalute error interpolation
    ynew1_upper_err = f1_upper_err(xnewCommon)
    ynew1_lower_err = f1_lower_err(xnewCommon)
    ynew1_upper_err_abs = np.abs(ynew1 - ynew1_upper_err)
    ynew1_lower_err_abs = np.abs(ynew1 - ynew1_lower_err)
    ynew1_err = np.mean([ynew1_upper_err_abs, ynew1_lower_err_abs], axis=0) # pairwise mean 
    
    # Compute chi2
    chi2 = np.sum(((ynew1 - ynew2) / ynew1_err)**2)
    return chi2
model_chi2.errordef = 1
    
# Init value
A_init = (peak1 / peak2 - 1 ) * c # shift between the two peaks

# Compute bounds on A
x1_min, x1_max = min(x1), max(x1)
A_lower_bound = (x1_min / peak2 - 1 ) * c
A_upper_bound = (x1_max / peak2 - 1 ) * c

minuit = Minuit(model_chi2, A=A_init)
minuit.limits["A"] = (A_lower_bound, A_upper_bound)
minuit.migrad()

# Results
valid = minuit.valid
shift_min_final = minuit.values['A']            # value used
shift_min_final_err = minuit.errors['A']        # error used

forced = minuit.fmin.has_made_posdef_covar
at_limit = minuit.fmin.has_parameters_at_limit

if forced or at_limit:
    valid = False

#### Finding overall difference per match

In [ ]:
def weighted_mean(x, errors):
    m1 = np.sum([x/s**2 for x, s in zip(x, errors)])
    m2 = np.sum([1/(x**2) for x in errors])
    mean = m1/m2
    err = np.sqrt(1/np.sum([1/(x**2) for x in errors]))
    return (mean, err)


# shifts, shifts_err are list of results for all matches in one observation comparison
shift_mean, shift_mean_err = weighted_mean(shifts, shifts_err)
median = np.median(shifts)

# but I use 
median, shift_mean_err
# and not the weighted mean, because it is influced my outliers too much. What is the error on the median? 

These values _median_ and _shift_mean_err_ are then put into the matrix for all observation comparisons (better word?)

#### Matrix reduction 

In [ ]:
def model_chi2(*V):
    V = np.asarray([*V])
    res = []
    size = diff_matrix.shape[0] 
    for x in np.arange(size):
        # for y in np.arange(x, size - 1):
        for y in np.arange(x, size):
            if x != y:
                diff_matrix[x, y]
                V[x]
                V[y]
                res.append(((diff_matrix[x, y] - (V[x] - V[y])) / diff_matrix_err[x, y])**2)
    chi2 = np.sum(res)
    return chi2
model_chi2.errordef = 1

# Use list of zeros as init values (len of the number of observations)
init_values = np.zeros(diff_matrix.shape[0])

minuit = Minuit(model_chi2, *init_values)
m = minuit.migrad()

final_shifts = minuit.values[:]             # final value
final_shifts_err = minuit.errors[:]         # final error